In [2]:
# dataset location
train_dataset_path = '/home/arka/Desktop/assignments/ic272/Assignment5/cifar-3class-data/train'
test_dataset_path = '/home/arka/Desktop/assignments/ic272/Assignment5/cifar-3class-data/test'

In [ ]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split

In [23]:
classes = ['aeroplane', 'car', 'bird']
def import_from_directory(_path):
    images = []

    for idx, lbl in enumerate(classes):
        path = os.path.join(_path, str(idx))
        for img in os.listdir(path):
            im_path = os.path.join(path, img)
            img = np.asarray(Image.open(im_path).convert('L'))
            images.append(img)
            
    images = np.array(images)
    count = images.shape[0]//3
    labels = np.array([0]*count + [1]*count + [2]*count)
    return images, labels
# dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

In [ ]:
images = np.random.randint(0, 1501, 4)
images

In [25]:
train_images, train_labels = import_from_directory(train_dataset_path)

In [ ]:
# plot 4 random images
fig, ax = plt.subplots(1, 4, figsize=(20, 20))
for i in range(4):
    ax[i].imshow(train_images[images[i]])
    ax[i].set_title(classes[train_labels[images[i]]])
    ax[i].axis('off')
plt.show()

In [24]:
im_train, im_cv, lbl_train, lbl_cv = train_test_split(train_images, train_labels, test_size=0.1)

In [28]:
print(f'''
Train ds size: Images {im_train.shape}; Labels {lbl_train.shape}
CV ds size: Images {im_cv.shape}; Labels {lbl_cv.shape}
''')


Train ds size: Images (1350, 32, 32); Labels (1350,)
CV ds size: Images (150, 32, 32); Labels (150,)



In [10]:
# flatten images
im_train = im_train.reshape(1350, 1024)
im_cv = im_cv.reshape(150, 1024)

In [11]:
# normalise
im_train = im_train/255
im_cv = im_cv/255

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024,)),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=3, activation='softmax')
])

In [ ]:
model.summary()

In [14]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


In [ ]:
history = model.fit(im_train, lbl_train, 
                    epochs=500, 
                    batch_size=200,
                    validation_data=(im_cv, lbl_cv))

In [ ]:
history.history.keys()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].plot(history.history['loss'], label='train loss')
ax[0].plot(history.history['val_loss'], label='cv loss')
ax[0].set_title('Loss vs epochs')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epochs')
ax[0].legend()

ax[1].plot(history.history['sparse_categorical_accuracy'], label='train accuracy')
ax[1].plot(history.history['val_sparse_categorical_accuracy'], label='cv accuracy')
ax[1].set_title('Accuracy vs epochs')
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epochs')
ax[1].legend()

plt.show()

In [19]:
model.save_weights(filepath='/home/arka/Desktop/assignments/ic272/Assignment5/weights_fcc.weights.h5')

In [29]:
test_images, test_labels = import_from_directory(test_dataset_path)

In [30]:
test_images.shape

(300, 32, 32)

In [31]:
test_images = test_images.reshape(300, 1024)
test_images = test_images/255

In [36]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5836 - sparse_categorical_accuracy: 0.6538 


In [38]:
print(f'Test loss: {test_loss:.3f}\nTest accuracy: {test_accuracy:.3f}')

Test loss: 2.271
Test accuracy: 0.690
